# Figure 3: illustrating the IBCM model in a simple case
This version is with a three-odor, slightly non-Gaussian background. Each concentration is given by $\nu = \nu_0 + \tilde{\nu} + \epsilon \tilde{\nu}^2$, where $\tilde{\nu}$ is a zero-mean Ornstein-Uhlenbeck process. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import os, colorsys, json

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
# Resources
data_folder = os.path.join("..", "results", "for_plots")
panels_folder = "panels/"
params_folder = os.path.join("..", "results", "common_params")

# Aesthetic parameters

In [ ]:
# rcParams
with open(os.path.join(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(os.path.join(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

In [ ]:
_, n_neu, n_orn = np.load(os.path.join("..", "results", "for_plots", "sample_non-gaussian_simulation.npz"))["mbarser"].shape

In [ ]:
# Play around with palettes
test_palette = sns.cubehelix_palette(n_colors=n_orn, 
        start=0.0, rot=1.0, gamma=1.0, hue=0.8, light=0.85, dark=0.15, reverse=True)
sns.palplot(test_palette)
plt.show()
plt.close()

fig, ax = plt.subplots()
for i in range(5):
    ax.plot(np.arange(100), np.random.normal(size=100)+(i-2)**2*0.5, color=test_palette[i*6])
plt.show()
plt.close()

In [ ]:
# Extra aesthetic parameters for this figure

# More legend rcParams: make everything smaller by 30 %
plt.rcParams["patch.linewidth"] = 0.75
legend_rc = {"labelspacing":0.5, "handlelength":2.0, "handleheight":0.7, 
             "handletextpad":0.8, "borderaxespad":0.5, "columnspacing":2.0}
for k in legend_rc:
    plt.rcParams["legend."+k] = 0.75 * legend_rc[k]
    
# Neuron color palettes
#neuron_colors = np.asarray(sns.color_palette("husl", n_colors=24))[[8, 16]]
neuron_colors = np.asarray(sns.husl_palette(n_colors=24, h=0.01, s=0.9, l=0.4, as_cmap=False))[[8, 17]]
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=24, h=0.01, s=0.9, l=0.4, as_cmap=False))
#neuron_colors = np.asarray(sns.husl_palette(n_colors=2, h=0.01, s=0.9, l=0.4, as_cmap=False))
# Make them all darker
#neuron_colors = np.asarray(sns.color_palette("cubehelix", n_colors=24))[[4, 12]]

new_color = "r"
linestyles = ["-", "--", ":", (0, (5, 1, 2, 1)), "-."]
#orn_colors = sns.color_palette("cubehelix", n_colors=n_orn)
orn_colors = sns.cubehelix_palette(n_colors=n_orn, 
        start=0.0, rot=1.0, gamma=1.0, hue=0.8, light=0.85, dark=0.15, reverse=True)

neuron_styles = linestyles + [(0, (1, 2, 1, 2))]
sns.palplot(neuron_colors)
print([mpl.colors.to_hex(a) for a in neuron_colors])

In [ ]:
# Save the ORN color map and the neuron color map to disk
with open(os.path.join(params_folder, "orn_colors.json"), "w") as f:
    json.dump(orn_colors, f)

with open(os.path.join(params_folder, "inhibitory_neuron_two_colors.json"), "w") as f:
    json.dump(neuron_colors.tolist(), f)

with open(os.path.join(params_folder, "inhibitory_neuron_full_colors.json"), "w") as f:
    json.dump(neuron_colors_full.tolist(), f)

# Panel A: background process
Choice 1: plot the three concentrations, margin histogram of one. 

Choice 2: plot one of the three concentrations (since all i.i.d.)

Choice 3: plot the 25 ORNs activities. Looks more complicated! Need 25 colors, ugly palette. Also unclear this is a low-dimensional/sparse background. 

In [ ]:
# Load example concentration time series
ex = np.load(os.path.join(data_folder, "sample_non-gaussian_simulation.npz"))
tser_example = ex["tser"]  # Unskip this
nuser_noskp = ex["nuser"]  # This has all time steps, for nicer histogram
epsilon_nu = ex["epsilon"]
averages_nu = ex["averages_nu0"]
conc_ser_noskp = averages_nu + nuser_noskp + epsilon_nu*nuser_noskp**2
skp_example = ex["skp"]
tser_noskp = np.arange(tser_example[0], tser_example[-1]+np.mean(np.diff(tser_example)), 
                               (tser_example[1]-tser_example[0])/skp_example)
nuser_example = nuser_noskp[::skp_example]
conc_ser_example = conc_ser_noskp[::skp_example]
n_components = averages_nu.shape[0]
back_components = ex["back_vecs"]

# Time units per simulation step, for plotting, in ms
dt_u = 10.0  # ms

In [ ]:
# Choice 3: plot time series and marginal histogram of all ORNs
# Compute ORN time series as A.\nu, where A is the matrix of background odor constant vectors. 
orn_ser_noskp = conc_ser_noskp.dot(back_components)

fig = plt.figure()
gs = fig.add_gridspec(1, 3)
axes = [fig.add_subplot(gs[:2])]
axes.append(fig.add_subplot(gs[2], sharey=axes[0]))
# Plot time series
t_window = slice(25000, 25200, 1)
tser_window = tser_noskp[t_window]
for i in range(0, n_orn, 6):
    clr = orn_colors[(i+6) % n_orn]
    axes[0].plot((tser_window - tser_window[0])*dt_u/1000, orn_ser_noskp[t_window, i], 
        label=str(i), color=clr, lw=0.8, alpha=0.8)
    # Plot a small histogram
    counts, bins = np.histogram(orn_ser_noskp[:, i], bins="sqrt", density=True)
    axes[1].barh(bins[:-1], counts, align="edge", height=bins[1:] - bins[:-1], 
                color=clr, alpha=0.5)
axes[0].set_xlabel("Time (s)")
axes[0].set_ylabel(r"OSN activity $s^i(t)$", labelpad=0.0)
axes[0].set_xticks(np.arange(0.0, 2.5, 0.5))
axes[0].set_ylim([-0.1, np.amax(orn_ser_noskp[t_window])*1.2])
axes[0].legend(title="OSN index", ncol=2, fontsize=6, frameon=False)
axes[1].set(xlabel="Prob. density", xticks=[0, 4, 8, 12])
#ax.tick_params(axis="x", which="major", pad=1.0)
[label.set_visible(False) for label in axes[1].get_yticklabels()]
fig.tight_layout()
#fig.savefig(os.path.join(panels_folder, "sample_non-gaussian_orn_series.pdf"),
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel B: $M$ series
Really, show in terms of the dot products with $\mathbf{x}_a$ and $\mathbf{x}_b$. Need a custom legend with colors and styles. 

In [ ]:
back_vecs_example = ex["back_vecs"]
n_b = back_vecs_example.shape[0]
mbarser_example = ex["mbarser"]
cbarser_example = ex["cbarser"]
yser_example = ex["sser"]
wser_example = ex["wser"]
saddle_h = ex["saddle_h"]
cgammaser_example = mbarser_example.dot(back_vecs_example.T)
n_i_ibcm = mbarser_example.shape[1]

# Analytical predictions
analytical_w = ex["analytical_w"]
analytical_cs_cn = ex["cs_cn"]
specif_gammas = ex["specif_gammas"]

# Matrix of all dot products for each neuron with each background odor
analytical_cgammas = np.ones([n_i_ibcm, n_b]) * analytical_cs_cn[1]
analytical_cgammas[np.arange(n_i_ibcm), specif_gammas] = analytical_cs_cn[0]
print(analytical_cgammas)

In [ ]:
# Choice 2: show two neurons
fig, ax = plt.subplots()
#ax.axhline(0.0, ls="-", color=(0.8,)*3, lw=0.8)
t_axis = tser_example*dt_u/1000/60
legend_styles = [[0, 0, 0], [0, 0, 0]]
i_highlights = [1, 3]  # Neurons to highlight
clr_back = back_palette[-1]
plot_skp = 10

# plot all other neurons first, skip some points
for i in range(n_i_ibcm):
    if i in i_highlights: 
        continue
    else: 
        for j in range(n_b):
            ax.plot(t_axis[::plot_skp], cgammaser_example[::plot_skp, i, j], color=clr_back, 
                ls="-", alpha=1.0-0.2*j, lw=plt.rcParams["lines.linewidth"]-j*0.3)

# Now plot the highlighted neuron
for j in range(n_b):
    for i in range(len(i_highlights)):
        li, = ax.plot(t_axis[::plot_skp], cgammaser_example[::plot_skp, i_highlights[i], j], 
                      color=neuron_colors[i], ls="-", alpha=1.0-0.2*j, 
                      lw=plt.rcParams["lines.linewidth"]-j*0.3)
        legend_styles[i][j] = li

# Annotate with analytical results
for j in range(2):
    ax.axhline(analytical_cs_cn[j], lw=1.0, ls="-.", color="k")
#ax.axhline(saddle_h, xmax=0.4, lw=1.0, ls=":", color="xkcd:magenta", alpha=0.3)
ax.annotate(r"Analytical $\bar{h}_{\mathrm{ns}}$", xy=(t_axis[-5], analytical_cs_cn[1]-0.4), 
            ha="right", va="top", color="k", size=6)
ax.annotate(r"Analytical $\bar{h}_{\mathrm{s}}$", xy=(t_axis[-5], analytical_cs_cn[0]+0.4), 
            ha="right", va="bottom", color="k", size=6)
ax.set(xlabel="Time (min)", 
       ylabel=r"Alignments $\bar{h}^i_{\gamma} = \mathbf{\bar{m}}^i \cdot \mathbf{s}_{\gamma}$")

# Need to do tight_layout before adding inset
fig.tight_layout()

# Custom legend: 2x3 table here, inset axes
axi = inset_axes(ax, width="50%", height="30%",
                   bbox_to_anchor=(.65, .42, .6, .5),  # left, bottom, width, height
                   bbox_transform=ax.transAxes, loc=3)
axi.tick_params(labelleft=True, labelbottom=False, labeltop=True)
for i in range(len(i_highlights)):
    for j in range(n_b):
        li = legend_styles[i][j]
        axi.plot([0.0+j, 0.5+j], [0.8*i, 0.8*i], color=li.get_color(), 
                 ls=li.get_linestyle(), alpha=li.get_alpha(), lw=li.get_linewidth())
for side in ["bottom", "left", "top", "right"]:
    axi.spines[side].set_visible(False)
axi.tick_params(axis="both", length=0, pad=5)
axi.set_xticks([0.25, 1.25, 2.25])
#axi.set_xticklabels([r"$\mathbf{\bar{m}}^i \cdot \mathbf{x}_a$", r"$\mathbf{\bar{m}} \cdot \mathbf{x}_b$"])
axi.set_xticklabels([1, 2, 3])
axi.set_xlabel(r"Component $\gamma$", size=6, labelpad=3)
axi.xaxis.set_label_position('top') 
axi.set_yticks([0, 0.8])
axi.invert_yaxis()
axi.set_yticklabels(["$i={}$".format(i) for i in i_highlights])
#axi.set_yticklabels(["Neuron 2", "Neuron 1"])
axi.set_ylabel("Neuron", size=6, labelpad=3)
    
fig.savefig(os.path.join(panels_folder, "sample_non-gaussian_m_series.pdf"), 
           transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel D: $W$ series
From the neuron main colors, make palettes for the $n_{R}$ components of each column $\mathbf{w}$ of the matrix $W$, corresponding to the different IBCM neurons. 

In [ ]:
# Choice 2: show two neurons
plot_skp = 10
t_axis = tser_example*dt_u/1000/60
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
axes = axes.flatten()
# Make a palette for each neuron
neuron_comp_palettes = [sns.light_palette(a, n_colors=n_orn) for a in neuron_colors]
analytic_colors = [a[n_orn//2] for a in neuron_comp_palettes]
j_highlights = i_highlights
for i in range(n_orn):
    for j in range(len(j_highlights)):
        ax = axes[j]
        li, = ax.plot(t_axis[::plot_skp], wser_example[::plot_skp, i, j_highlights[j]], 
                      zorder=analytical_w.size + i*n_orn + j, 
                      color=neuron_comp_palettes[j][i], lw=0.8)
        lbl = "Analytical" if i == 0 else ""
        ax.axhline(analytical_w[i, j_highlights[j]], color=analytic_colors[j], ls="--", lw=0.65, label=lbl)
for j in range(len(j_highlights)):
    axes[j].set_title("Neuron $j = {}$".format(j_highlights[j]))
    axes[j].set_xlabel("Time (min)")
    axes[j].set_ylim(-0.05, axes[j].get_ylim()[1]*1.05)
    axes[j].set_xlim(axes[j].get_xlim()[0], axes[j].get_xlim()[1]*1.03)
    axes[j].legend(frameon=False)
    axes[j].set_xticks(np.arange(0, 50, 15))
axes[0].set_ylabel(r"Inhibitory weights ${W^i}_j$")
fig.tight_layout()
fig.savefig(os.path.join(panels_folder, "sample_non-gaussian_w_series.pdf"), transparent=True, 
           bbox_inches="tight")
plt.show()
plt.close()

# Panel E: background inhibition
Which color set to use? The model colors (IBCM, BioPCA, average subtraction, etc.). So IBCM here is turquoise. 

Which metric to plot? I think we should just plot the activity norm, $\sqrt{\mathbf{s}^T \mathbf{s}}$
Which metric to plot? I think we should just plot $\sqrt{\langle \mathbf{s}^T \mathbf{s} \rangle}$, the RMS norm, compared to the background's, $\sqrt{\langle \mathbf{x}_B^T \mathbf{x}_B \rangle}$, to illustrate both the reduction in the average and the reduction in the fluctuations. This is better than plotting the average norm ($\langle \sqrt{\mathbf{s}^T \mathbf{s}} \rangle$). 

In [ ]:
models = ["ibcm", "biopca", "avgsub", "optimal", "ideal", "orthogonal", "none"]
model_nice_names = {
    "ibcm": "IBCM",
    "biopca": "BioPCA",
    "avgsub": "Average",
    "ideal": "Ideal",
    "optimal": "Optimal",
    "orthogonal": "Orthogonal",
    "none": "None"
}
model_colors = {
    "ibcm": "xkcd:turquoise",
    "biopca": "xkcd:orangey brown",
    "avgsub": "xkcd:navy blue",
    "optimal": "xkcd:powder blue",
    "ideal": "xkcd:light green",
    "orthogonal": "xkcd:pale rose",
    "none": "grey"
}

In [ ]:
analytical_w.dot(analytical_cgammas).shape

In [ ]:
# Compute background series, s series, norm series, etc. 
bkvecser_example = conc_ser_example.dot(back_components)
norm_y2 = np.sum(yser_example**2, axis=1)
norm_x2 = np.sum(bkvecser_example**2, axis=1)

# Analytical prediction of vec{s} norm: use analytic W, compute analytic s, compute its norm
yser_analytic = bkvecser_example - conc_ser_example.dot((analytical_w.dot(analytical_cgammas)).T)
print(yser_analytic.shape)
norm_y2_analytic = np.sum(yser_analytic**2, axis=1)
norm_y_analytic_mean = np.mean(np.sqrt(norm_y2_analytic), axis=0)
variance_y_analytic = np.mean(norm_y2_analytic, axis=0) - np.mean(np.sqrt(norm_y2_analytic), axis=0)**2
std_y_analytic = np.sqrt(variance_y_analytic)

In [ ]:
fig, ax = plt.subplots()
plot_skp = 1
ax.plot(t_axis[::plot_skp], np.sqrt(norm_x2[::plot_skp]), 
        color=back_color_samples, label=r"Background $\|\mathbf{s}_\mathrm{B}\|$", lw=0.5)
ax.plot(t_axis[::plot_skp], np.sqrt(norm_y2[::plot_skp]), 
        color=model_colors["ibcm"], label=r"IBCM $\|\mathbf{y}\|$", lw=0.5)
ax.set(xlabel="Time (min)", ylabel=r"PN activity norm, $\|\mathbf{y}\|$")
ax.set_xticks(np.arange(0.0, t_axis[-1], 10.0))
ax.axhline(norm_y_analytic_mean+std_y_analytic, ls="-.", label=r"Analytical $\langle \|\mathbf{y}\| \rangle$",
           color=(mpl.colors.to_rgba(model_colors["ibcm"]) + np.asarray([0.25,]*3 + [0])).clip(0, 1))
#ax.plot(t_axis[::plot_skp], np.sqrt(norm_y2_analytic[::plot_skp]), 
#        ls="-", label=r"Analytical $\|\mathbf{s}\|$",
#        color=(mpl.colors.to_rgba(model_colors["ibcm"]) + np.asarray([0.25,]*3 + [0])).clip(0, 1))
ax.set_ylim([ax.get_ylim()[0], ax.get_ylim()[1]*1.4])
ax.legend(bbox_to_anchor=(0.98, 1.0), loc="upper right", frameon=False, fontsize=6)
fig.tight_layout()
fig.savefig(os.path.join(panels_folder, "sample_non-gaussian_s_series.pdf"), 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
for i in range(n_orn):
    print(i, ":", np.corrcoef(yser_example[:, i], yser_analytic[:, i])[0, 1])